In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./list 2.csv',sep=',')
df = df.dropna()

#기업명
df["기업명"] = df["기업명"].apply(lambda x: re.sub('[|0-9기]', '', x))      # ‘| 1기’ 와 같은 기수 제외

#관심분야 더미변수화


interests = ['경영','컨설팅','마케팅','경제','금융','과학','공학','기술','IT',
             '교육','디자인','사진','예술','영상','콘텐츠','문화','역사','뷰티','미용',
             '화장품','사회공헌','교류','언론','미디어','여행','호텔','항공','요리','식품',
             '의료','보건','정치','법률','창업','자기계발','행사','페스티벌','환경',
             '에너지','기타','체육','헬스']                                                                  #위 키워드로 단어 정제
new_df = df.copy()                                                                                          #새로운 DataFrame 생성, 덮어쓰기 방지

for interest in interests:
    new_df['관심분야'] = new_df['관심분야'].apply(lambda x: '/'.join([re.sub(f'{interest}.+', interest, i) for i in x.split('/')]))
     # 위 '키워드'+@ 내용들 '키워드'로 교체 예)마케팅,마케팅창업,마케팅경제,마케팅행사 => 모두 '마케팅'으로 변경
df = new_df

df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('경영/컨설팅/마케팅', '경영컨설팅마케팅') if x.startswith('경영/컨설팅/마케팅') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('과학/공학/기술/IT', '과학공학기술IT') if x.startswith('과학/공학/기술/IT') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('경제/금융', '경제금융') if x.startswith('경제/금융') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('디자인/사진/예술/영상', '디자인사진예술영상') if x.startswith('디자인/사진/예술/영상') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('문화/역사', '문화역사') if x.startswith('문화/역사') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('뷰티/미용/화장품', '뷰티미용화장품') if x.startswith('뷰티/미용/화장품') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('사회공헌/교류', '사회공헌교류') if x.startswith('사회공헌/교류') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('언론/미디어', '언론미디어') if x.startswith('언론/미디어') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('여행/호텔/항공', '여행호텔항공') if x.startswith('여행/호텔/항공') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('요리/식품', '요리식품') if x.startswith('요리/식품') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('의료/보건', '의료보건') if x.startswith('의료/보건') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('정치/사회/법률', '정치사회법률') if x.startswith('정치/사회/법률') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('창업/자기계발', '창업자기계발') if x.startswith('창업/자기계발') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('행사/페스티벌', '행사페스티벌') if x.startswith('행사/페스티벌') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('환경/에너지', '환경에너지') if x.startswith('환경/에너지') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('체육/헬스', '체육헬스') if x.startswith('체육/헬스') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('기타', '기타') if x.startswith('기타') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: x.replace('콘텐츠', '콘텐츠') if x.startswith('콘텐츠') else x)
df['관심분야'] = df['관심분야'].apply(lambda x: '교육' if x.startswith('교육') else x.split('/')[0])        #교육 뒤에오는/모두 제거

# 모든 분야를 더미변수화
dummy = df['관심분야'].str.get_dummies(sep='/')                                     #17개 더미변수

# 더미변수화한 결과와 원래 데이터프레임을 합치기
df = pd.concat([df, dummy], axis=1)
df['기타체육헬스'] = pd.concat([df['체육헬스'], df['기타']], axis=1).sum(axis=1)                                #체육헬스 열과 기타 열 합치기
df = df.drop(['체육헬스', '기타'], axis=1)                                                                      #체육헬스,기타 열 삭제
df['디자인사진예술영상콘텐츠'] = pd.concat([df['디자인사진예술영상'], df['콘텐츠']], axis=1).sum(axis=1)           #디자인사진예술영상 열과 콘텐츠열 합치기
df = df.drop(['디자인사진예술영상', '콘텐츠'], axis=1)                                                              #디자인사진예술영상 열, 콘텐츠열 삭제


#우대역량 더미변수화
df["우대역량"] = df["우대역량"].str.replace(", 기타", "")                                                       #',기타'가 규칙없이 붙어있는경우가 많아서 삭제
df['우대역량'] = df['우대역량'].replace('기타', '우대역량_기타')                                                 #'기타'열이 관심분야에도 중복됨으로 교체 
df = df.join(df["우대역량"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#모집인원
df["모집 인원"]=df["모집인원"].str.extract(r'(\d+)')     #숫자만 남기기
df=df.replace({"모집 인원":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"모집 인원":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"모집 인원":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"모집 인원":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"모집 인원":'int'})                   
student_bins=[0,1,21,101,50000]                        #범주화  0:인원미정/1-20(명):소/21-100(명):중/101-50000(명):대
student_scale=["인원미정","소","중","대"]               
df = pd.concat([df, pd.cut(df["모집 인원"], bins=student_bins, right=False, labels=student_scale).rename("모집인원규모")], axis=1) #분포범주화

#열삭제
df=df.drop(["대외활동명","접수기간","활동지역","활동기간"],axis='columns')

#열위치조정
df = df.reindex(columns=["인덱스","기업명","관심분야","경영컨설팅마케팅","경제금융","과학공학기술IT",
                         "교육","문화역사","뷰티미용화장품","사회공헌교류","언론미디어","여행호텔항공",
                         "요리식품","의료보건","정치사회법률","창업자기계발","행사페스티벌","환경에너지",
                         "기타체육헬스","디자인사진예술영상콘텐츠","우대역량","리더십","멘토링/수업 진행 경험",
                         "모두 지원 가능","사진/영상/디자인 툴","외국어","우대역량_기타","취재/인터뷰/기사 경험",
                         "컴퓨터 활용능력","콘텐츠 기획/제작 경험","타 대외활동 경험","파워블로거/SNS",
                         "행사 기획 경험","휴학 중/시간 투자 가능","모집 인원","모집인원규모"
                         ])

#데이터저장
df=df.to_csv("activity.csv")
